# Creazione matrici png

In [ ]:
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw

import random,string
import os, sys, time

# font = ImageFont.truetype("Arial-Bold.ttf",14)
font = ImageFont.truetype("arial.ttf", 100) #100 è la size

N_SAMPLES = 10000 #numero captcha completi
N_SMP_LETTERE_DEFORMATE =  26000



In [ ]:
#creazione cartelle

if not os.path.exists("./matrici/"):
    os.makedirs("./matrici/")
if not os.path.exists("./matrici/originale/"):
    os.makedirs("./matrici/originale/")
if not os.path.exists("./captcha/completi/"):
    os.makedirs("./captcha/completi/")
if not os.path.exists("./captcha/singoli/"):
    os.makedirs("./captcha/singoli/")

In [ ]:
size = 200 #px

def creaMatrice(lettera):
    img=Image.new("1", (size,size),color=1)
    draw = ImageDraw.Draw(img)
    draw.text((50, 50),lettera,fill=0,font=font)
    draw = ImageDraw.Draw(img)
    #noisy(img,size)
    img.save(f"./matrici/originale/{lettera}.png")
    


Funzione di add noise per i campioni

In [ ]:
# Funzione di noise per i campioni
def noisy(img,width,height):
    draw = ImageDraw.Draw(img)
    for x in range(0,15):
        draw.point((random.randint(0,width),random.randint(0,height)),0)
        draw.line([(random.randint(0,width),random.randint(0,height)), (random.randint(0,width),random.randint(0,height))],width=3)
        #linee verticali
        xy = (random.randint(0,width),random.randint(0,height))
        xy_due = (random.randint(xy[0]-50,xy[0]+50) ,random.randint(0,height))
        draw.line([xy,xy_due],width=3)
    return img

Creazione matrici

In [ ]:
#creazione matrici
alfabeto = list(string.ascii_uppercase) #or list(map(chr, range(ord('a'), ord('z')+1)))

#alfabeto = ["A","B","C","D","E"]
for l in alfabeto:
    creaMatrice(l)

In [ ]:
#immagine di esempio
Image.open(os.path.join("./matrici/originale","A.png"))

# Augmentor

In [ ]:
import Augmentor
p = Augmentor.Pipeline("./matrici/originale/")

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.random_distortion(probability=1, grid_width=5, grid_height=5, magnitude=8)

p.skew(probability=1)

In [ ]:
print(N_SMP_LETTERE_DEFORMATE)

In [ ]:
p.sample(N_SMP_LETTERE_DEFORMATE)

In [ ]:
#immagine di esempio deformata
Image.open(os.path.join("./matrici/originale/output","originale_original_A.png_0074a92e-09fd-4850-8249-bca7d04fc62d.png")) 

# Generazione di captcha random

In [ ]:
def getfirstUppercase(s):
    return ([char for char in s if char.isupper()])[0]

In [ ]:
path = "./matrici/originale/output/" #sono le matrici distorte, 200x200
dirs = os.listdir( path ) #converte il contenuto della directory in una lista

#intervallo random lunghezza captcha
MAX_IMAGES=7
MIN_IMAGES=5

#altezza
max_height = 200

#generazione
for v in range (0,N_SAMPLES):
    num_images = random.randint(MIN_IMAGES,MAX_IMAGES) #lunghezza captcha in lettere
    total_width = num_images*200 #lunghezza immagine in px
    
    new_im = Image.new("1", (total_width, max_height))
    filename = ""
    x_offset = 0
    list_images = []
    for i in range (0,num_images): #num_images elementi
        curr_rand=random.randint(0,len(dirs)-1)
        list_images.append(os.path.join(path,dirs[curr_rand]))
        valore = dirs[curr_rand] #nome file random
        filename+=getfirstUppercase(valore) #si aggiunge al filename la lettera corripondente

    for im in list_images:
        image_cur = Image.open(im)
        new_im.paste(image_cur, (x_offset,0))
        x_offset += image_cur.size[0]
    #sporca new_im
    new_im = noisy(new_im,width=total_width,height=max_height)
    #salva new_im
    new_im.save(f'./captcha/completi/{filename}{v}.png') # es stringa AEDTG45.png

In [ ]:
#Croppa tutti i captcha e smista le lettere nelle cartelle
path = "./captcha/completi/"
dirs = os.listdir( path )

for item in dirs:
    if os.path.isfile(path+item):
        lenght_img = int(Image.open(path+item).size[0] / 200)
        for i in range(0,lenght_img):
            im = Image.open(path+item)
            #crop ogni 200px lenght_img volte
            #0,0 200,200 - 200,0 400,200 - 400,0 600, 200
            current_box = im.crop((200*i,0,200*(i+1),200))

            #legge la lettera corrispondente
            if not os.path.exists("./captcha/singoli/"+item[i]):
                os.makedirs("./captcha/singoli/"+item[i])
            #salva nella cartella corrispondente
            current_box.save(f"./captcha/singoli/{item[i]}/{i}{time.time()}.png")
            


# Organizzazione files

Il numero di campioni nelle varie cartelle viene uniformato con il numero di campioni del meno frequente

In [ ]:
import os, os.path

In [ ]:
path = "./captcha/singoli/"
dirs = os.listdir( path )
smp_num = []

#List number of elements per folder
for item in dirs:
    folder = ([(f) for f in os.listdir(path+item) if os.path.isfile(os.path.join(path,item,f))])
    old_lenght = len(folder)
    smp_num.append((len([f for f in os.listdir(path+item) if os.path.isfile(os.path.join(path,item,f))])))
    print(item, old_lenght)

minimum = min(smp_num)
print("The minimum is",minimum)

In [ ]:
smp_num=[]

#Cancella il numero di elementi in eccesso per cartella
for item in dirs:
    folder = ([(f) for f in os.listdir(path+item) if os.path.isfile(os.path.join(path,item,f))])
    old_lenght = len(folder)
    
    smp_num.append((len([f for f in os.listdir(path+item) if os.path.isfile(os.path.join(path,item,f))])))
    
    #per ogni folder, rimuovi len(folder) - minimum elementi
    times = (len(folder)-minimum)
    for i in range(0,times):
        os.remove(os.path.join(path, item ,folder[i]))
        folder.pop(i)
    
    new_lenght = len(folder)
    print(item,old_lenght, "->", new_lenght)
    

In [ ]:
#Split library
import split_folders

In [ ]:
input_folder = path
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio(input_folder, output="output", ratio=(.6, .2, .2)) 

# Generazione di captcha custom (opzionale)

In [ ]:
parola = list("UNICT")

save_path = "./CaptchaTest/"

path = "./matrici/originale/output/" 
dirs = os.listdir( path )

num_images = len(parola)

max_height = 200
total_width = num_images*200


new_im = Image.new("1", (total_width, max_height))
filename = ""
x_offset = 0
list_images = []

matrici = []

for l in parola:
    print(str(l))
    for k in dirs:
        if getfirstUppercase(k) == l: #è buono e lo prendo
            #print(k)
            matrici.append(k)
    curr_rand=random.randint(0,len(matrici)-1)
    list_images.append(path+matrici[curr_rand])
    matrici=[]
            

for im in list_images:
    image_cur = Image.open(im)
    new_im.paste(image_cur, (x_offset,0))
    x_offset += image_cur.size[0]

new_im = noisy(new_im,width=total_width,height=max_height)
new_im.save(f'{save_path}{"".join(parola)}{random.randint(0,500)}.png') # es stringa AEDTG45.png